In [0]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .appName("ML Model")
         .getOrCreate())

sc = spark.sparkContext

In [0]:
# View the mounted file system
display(dbutils.fs.ls("/mnt/my_data"))
display(dbutils.fs.ls("/mnt/my_data/ml_training"))


path,name,size,modificationTime
dbfs:/mnt/my_data/Posts/,Posts/,0,1715602088000
dbfs:/mnt/my_data/Users/,Users/,0,1715151475000
dbfs:/mnt/my_data/ml_training/,ml_training/,0,1715518261000
dbfs:/mnt/my_data/model/,model/,0,1715590248000
dbfs:/mnt/my_data/postTypes/,postTypes/,0,1715151456000
dbfs:/mnt/my_data/stringindexer/,stringindexer/,0,1715590289000


path,name,size,modificationTime
dbfs:/mnt/my_data/ml_training/PostTypes.txt,PostTypes.txt,126,1715518290000
dbfs:/mnt/my_data/ml_training/Posts/,Posts/,0,1715518366000
dbfs:/mnt/my_data/ml_training/users.csv,users.csv,176980816,1715518304000


In [0]:
posts = spark.read.parquet("/mnt/my_data/Posts/*")
ml_model = "/mnt/my_data/model"
stringindexer = "/mnt/my_data/stringindexer"

In [0]:
# User defined function
def predictions_udf(df, ml_model, stringindexer):
    from pyspark.sql.functions import col, regexp_replace, lower, trim
    from pyspark.ml import PipelineModel

    # Filter out empty body text
    df = df.filter("Body is not null")
    # Making sure the naming of the columns are consistent with the model
    df = df.select(col("Body").alias("text"), col("Tags"))
    # Preprocessing of the feature column
    cleaned = df.withColumn('text', regexp_replace('text', r"http\S+", "")) \
                    .withColumn('text', regexp_replace('text', r"[^a-zA-z]", " ")) \
                    .withColumn('text', regexp_replace('text', r"\s+", " ")) \
                    .withColumn('text', lower('text')) \
                    .withColumn('text', trim('text')) 

    # Load in the saved pipeline model
    model = PipelineModel.load(ml_model)

    # Making the prediction
    prediction = model.transform(df)

    predicted = prediction.select(col('text'), col('Tags'), col('prediction'))

    # Decoding the indexer
    from pyspark.ml.feature import StringIndexerModel, IndexToString

    # Load in the StringIndexer that was saved
    indexer = StringIndexerModel.load(stringindexer)

    # Initialize the IndexToString converter
    i2s = IndexToString(inputCol = 'prediction', outputCol = 'decoded', labels = indexer.labels)
    converted = i2s.transform(predicted)

    # Display the important columns
    return converted

In [0]:
result = predictions_udf(posts,ml_model, stringindexer)
# display(result)

In [0]:
# change the column name 
topics = result.withColumnRenamed('decoded', 'topic').select('topic')

# Aggregate the topics and calculate the total qty of each topic
topic_qty = topics.groupBy(col("topic")).agg(count('topic').alias('qty')).orderBy(desc('qty'))

In [0]:
display(topic_qty)

In [0]:
# define this function

def crt_sgl_file(result_path):
        # write the result to a folder container several files
        path = "/mnt/my_data/BI/ml_result"
        topic_qty.write.option("delimiter", ",").option("header", "true").mode("overwrite").csv(path)

        # list the folder, find the csv file 
        filenames = dbutils.fs.ls(path)
        name = ''
        for filename in filenames:
            if filename.name.endswith('csv'):
                org_name = filename.name

        # copy the csv file to the path you want to save, in this example, we use  "/mnt/deBDProject/BI/ml_result.csv"
        dbutils.fs.cp(path + '/'+ org_name, result_path)

        # delete the folder
        dbutils.fs.rm(path, True)

        print('single file created')

In [0]:
# run the function
result_path = "/mnt/my_data/BI/ml_result.csv"

crt_sgl_file(result_path)

single file created


In [0]:
display(result_path)

'/mnt/my_data/BI/ml_result.csv'

In [0]:
display(dbutils.fs.ls("/mnt/my_data/BI/ml_result.csv/"))

path,name,size,modificationTime
dbfs:/mnt/my_data/BI/ml_result.csv,ml_result.csv,1997,1715605207000
